# OMOP to Drugs For Treatments

As in from IDEA4RC to OMOP, this is not clear yet and it is hard to understand the mapping in: https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=634754097#gid=634754097 